In [2]:
# ----------------------------------------------------------------------
#  ems_optimization_day30.py  (25h + Curtailment + 00:00 SOC = 50%)
# ----------------------------------------------------------------------
import numpy as np
import pandas as pd
import pulp

# ----------------------------------------------------------------------
# 2. Load forecasts + CLAMP NEGATIVE PV TO ZERO
# ----------------------------------------------------------------------
solar_df = pd.read_csv('solar_forecast_day30.csv')
load_df  = pd.read_csv('load_forecast_day30.csv')

# FIX: Clamp negative solar to zero
solar_W = np.maximum(solar_df['solar_forecast'].values, 0)
load_W  = load_df['load_forecast'].values

print(f"Negative PV values clamped: {(solar_df['solar_forecast'].values < 0).sum()}")

# ----------------------------------------------------------------------
# 3. Convert to kWh
# ----------------------------------------------------------------------
pv_kWh   = solar_W / 1000.0
load_kWh = load_W  / 1000.0
T = 25  # 25 dispatch intervals (00:00–01:00, ..., 23:00–00:00 next day)

# ----------------------------------------------------------------------
# 4. Parameters
# ----------------------------------------------------------------------
batt_cap = 25.0
batt_eff = 0.95
soc_init = 0.50        # 50% at 00:00
soc_min  = 0.20
soc_max  = 0.80
diesel_rated = 15.0

# ----------------------------------------------------------------------
# 5. PuLP Model
# ----------------------------------------------------------------------
prob = pulp.LpProblem("Microgrid_EMS_Curtailment", pulp.LpMinimize)

# Variables (t=0 to 24 → 25 intervals)
batt_in   = pulp.LpVariable.dicts("batt_in",   range(T), lowBound=0, upBound=batt_cap)
batt_out  = pulp.LpVariable.dicts("batt_out",  range(T), lowBound=0, upBound=batt_cap)
diesel_E  = pulp.LpVariable.dicts("diesel_E",  range(T), lowBound=0, upBound=diesel_rated)
diesel_on = pulp.LpVariable.dicts("diesel_on", range(T), cat='Binary')
soc       = pulp.LpVariable.dicts("soc",       range(T), lowBound=soc_min*batt_cap, upBound=soc_max*batt_cap)
curtail   = pulp.LpVariable.dicts("curtail",   range(T), lowBound=0)

# ----------------------------------------------------------------------
# 6. Objective: Minimize diesel ON + curtailment
# ----------------------------------------------------------------------
prob += (
    1000 * pulp.lpSum(diesel_on[t] for t in range(T)) +
    1 * pulp.lpSum(curtail[t] for t in range(T))
)

# ----------------------------------------------------------------------
# 7. Energy balance
# ----------------------------------------------------------------------
for t in range(T):
    pv_used = pv_kWh[t] - curtail[t]

    prob += pv_used + diesel_E[t] + batt_out[t] / batt_eff == load_kWh[t] + batt_in[t]
    prob += pv_used >= 0
    prob += curtail[t] <= pv_kWh[t]
    prob += diesel_E[t] <= diesel_rated * diesel_on[t]
    prob += batt_in[t] + batt_out[t] <= batt_cap

# ----------------------------------------------------------------------
# 8. SOC dynamics
# ----------------------------------------------------------------------
# soc[0] = SOC at 01:00 (after first hour)
prob += soc[0] == soc_init * batt_cap + batt_eff * batt_in[0] - batt_out[0] / batt_eff

for t in range(1, T):
    prob += soc[t] == soc[t-1] + batt_eff * batt_in[t] - batt_out[t] / batt_eff

# ----------------------------------------------------------------------
# 9. Solve
# ----------------------------------------------------------------------
status = prob.solve(pulp.PULP_CBC_CMD(msg=False))
print("\n=== Solver status ===")
print(pulp.LpStatus[status])

if status != 1:
    raise RuntimeError("Infeasible — check load vs PV+storage")

# ----------------------------------------------------------------------
# 10. Extract results
# ----------------------------------------------------------------------
batt_in_vals   = [pulp.value(batt_in[t])   for t in range(T)]
batt_out_vals  = [pulp.value(batt_out[t])  for t in range(T)]
diesel_E_vals  = [pulp.value(diesel_E[t])  for t in range(T)]
diesel_on_vals = [pulp.value(diesel_on[t]) for t in range(T)]
soc_vals_kWh   = [pulp.value(soc[t])       for t in range(T)]
curtail_vals   = [pulp.value(curtail[t])   for t in range(T)]
soc_vals_pct   = np.array(soc_vals_kWh) / batt_cap * 100

total_diesel_h = sum(diesel_on_vals)
total_curtail_kWh = sum(curtail_vals)

print(f"\nDiesel ON: {total_diesel_h:.1f} hours")
print(f"Total PV Curtailed: {total_curtail_kWh:.2f} kWh")
print(f"Initial SOC (00:00): {soc_init*100:.1f}%")
print(f"First dispatch SOC (01:00): {soc_vals_pct[0]:.1f}%")
print(f"Final SOC (00:00 next day): {soc_vals_pct[24]:.1f}%")

# ----------------------------------------------------------------------
# 11. Save 26-row CSV: [00:00] + [01:00 to 00:00 next day]
# ----------------------------------------------------------------------
# Initial state at 00:00
init_row = {
    'batt_in_kWh'   : 0.0,
    'batt_out_kWh'  : 0.0,
    'diesel_kWh'    : 0.0,
    'diesel_on'     : 0.0,
    'pv_kWh'        : pv_kWh[0],
    'load_kWh'      : load_kWh[0],
    'curtail_kWh'   : 0.0,
    'soc_kWh'       : soc_init * batt_cap,
    'soc_pct'       : soc_init * 100
}

# Full dispatch (25 hours)
dispatch_rows = [
    {
        'batt_in_kWh'   : batt_in_vals[t],
        'batt_out_kWh'  : batt_out_vals[t],
        'diesel_kWh'    : diesel_E_vals[t],
        'diesel_on'     : diesel_on_vals[t],
        'pv_kWh'        : pv_kWh[t],
        'load_kWh'      : load_kWh[t],
        'curtail_kWh'   : curtail_vals[t],
        'soc_kWh'       : soc_vals_kWh[t],
        'soc_pct'       : soc_vals_pct[t]
    } for t in range(T)
]

# Combine: 00:00 + 25 dispatch steps → 26 rows
full_results = pd.DataFrame([init_row] + dispatch_rows)

full_results.to_csv('ems_decisions_25h.csv', index=False)
print("\n26-row dispatch saved: 00:00 (initial) + 25 hourly steps")
print("   → Row 0: 00:00 (SOC = 50%)")
print("   → Row 1: 01:00 (first dispatch)")
print("   → Row 25: 00:00 next day")

Negative PV values clamped: 7

=== Solver status ===
Optimal

Diesel ON: 3.0 hours
Total PV Curtailed: 0.92 kWh
Initial SOC (00:00): 50.0%
First dispatch SOC (01:00): 41.1%
Final SOC (00:00 next day): 20.0%

26-row dispatch saved: 00:00 (initial) + 25 hourly steps
   → Row 0: 00:00 (SOC = 50%)
   → Row 1: 01:00 (first dispatch)
   → Row 25: 00:00 next day


In [3]:
# Operational cost parameters (in ZAR)
c_pv = 0.15          # ZAR/kWh for PV O&M
c_batt = 1.8         # ZAR/kWh discharged for battery degradation
c_diesel_fuel = 7.0  # ZAR/kWh for diesel fuel
c_runtime = 100.0    # ZAR/hour for diesel runtime maintenance

# Calculate sums
total_pv_used = sum(pv_kWh[t] - curtail_vals[t] for t in range(T))
total_batt_out = sum(batt_out_vals)
total_diesel_e = sum(diesel_E_vals)
total_diesel_on = sum(diesel_on_vals)

# Total cost
total_cost = (c_pv * total_pv_used) + (c_batt * total_batt_out) + (c_diesel_fuel * total_diesel_e) + (c_runtime * total_diesel_on)

print(f"\nTotal Operational Cost: {total_cost:.2f} ZAR")


Total Operational Cost: 719.05 ZAR
